In [89]:
import numpy as np
import datetime
from collections import defaultdict
from argparse import Namespace
import json
import os
import copy
from shutil import copyfile
from scipy.spatial.transform import Rotation
#from pointcloud import translate_transform_to_new_center_of_rotation

In [87]:
def get_mat_angle(translation=None, rotation=None, rotation_center=np.array([0., 0, 0])):
    mat1 = np.eye(4)
    mat2 = np.eye(4)
    mat3 = np.eye(4)
    mat1[:3, 3] = -rotation_center
    mat3[:3, 3] = rotation_center
    if translation is not None:
        mat3[:3, 3] += translation
    if rotation is not None:
        mat2[:3, :3] = Rotation.from_rotvec(np.array([0, 0, 1.]) * rotation).as_dcm()
    return np.matmul(np.matmul(mat3, mat2), mat1)

In [2]:
def translate_transform_to_new_center_of_rotation(all_pred_translations, all_pred_angles, all_pred_centers, all_gt_pc1centers):
    new_all_pred_translations = np.zeros_like(all_pred_translations)
    for idx, (pred_translation, pred_angle, pred_center, gt_pc1center) in enumerate(zip(all_pred_translations, all_pred_angles, all_pred_centers, all_gt_pc1centers)):
        old_center, new_center = pred_center, gt_pc1center
        center_shift = new_center - old_center
        #  pred_transform = get_mat_angle(pred_translation, pred_angle, rotation_center=pred_center)
        #  new_pred_translation = pred_translation
        new_pred_translation = -center_shift + (np.matmul(get_mat_angle(rotation=pred_angle)[:3, :3], center_shift)) + pred_translation
        new_all_pred_translations[idx] = new_pred_translation
    return new_all_pred_translations

In [9]:
base_eval_dir = '/home/usuario/project_data/trained/KITTITrackletsCars/val/eval000038'

In [22]:
base_gt_dir = '/home/usuario/project_data/datasets/KITTITrackletsCars/'

In [70]:
val_idxs = np.loadtxt(base_gt_dir+'split/val.txt', dtype=int)

In [47]:
def to_arr(value):
    """ Converts a string with float values to an actual list of floats """
    return [float(name) for name in value.split()]

def fix_meta(meta):
    """ Converts all the string coded values to lists """
    meta['start_position'] = to_arr(meta['start_position'])
    meta['end_position'] = to_arr(meta['end_position'])
    meta['translation'] = to_arr(meta['translation'])
# Create dataset path
def load_all_metas(dataset_path):
    # Loads all jsons and stores them in a list(container)
    container = list()
    meta_path = os.path.join(base_gt_dir, "meta")
    for filename in sorted(os.listdir(meta_path)):
        # Create path and load file
        file_path = os.path.join(meta_path, filename)
        with open(file_path) as json_file: meta_dict = json.load(json_file)
        # Convert the string to lists
        fix_meta(meta_dict)
        meta_dict['filename'] = filename[:-5]
        # Append to file
        container.append(meta_dict)
    
    return container

In [48]:
metas = load_all_metas(base_gt_dir)

In [83]:
def get_gt(eval_metas):
    """  """
    centers = np.empty((len(val_idxs), 3), dtype=np.float32)
    translations = np.empty((len(val_idxs), 3), dtype=np.float32)
    angles = np.empty((len(val_idxs), 1), dtype=np.float32)

    for i, obs in enumerate(eval_metas):
        centers[i,:] = obs['start_position'] 
        translations[i,:] = obs['translation'] 
        angles[i,:] = obs['rel_angle']
    return {'centers': centers, 'translations': translations, 'angles': angles}

In [85]:
base_gt_dir = '/home/usuario/project_data/datasets/KITTITrackletsCars/'
val_idxs = np.loadtxt(base_gt_dir+'split/val.txt', dtype=int)    # List with all the values
# Load predictions
all_pred_translations = np.load(f'{base_eval_dir}/pred_translations.npy')
all_pred_angles = np.load(f'{base_eval_dir}/pred_angles.npy')
all_pred_centers = np.load(f'{base_eval_dir}/pred_s2_pc1centers.npy')

# Load ground truth
gt = get_gt(eval_metas)  
all_gt_pc1centers = gt['centers']
all_gt_translations = gt['translations']
all_gt_angles = gt['angles']

eval_dir="./"
accept_inverted_angle=False
detailed_eval=False
avg_window=5
mean_time=0


In [95]:
def ns_to_dict(ns):
    return {k: ns_to_dict(v) if type(v) == Namespace else v for k, v in ns.__dict__.items()}


def eval_translation(t, gt_t):
    levels = np.array([0, 0, 0])
    level_thresholds = np.array([0.02, 0.1, 0.2])
    dist = np.linalg.norm(t[:2] - gt_t[:2])
    for idx, thresh in enumerate(level_thresholds):
        if dist < thresh:
            levels[idx] = 1
    return dist, levels


def angle_diff(a, b):
    d = b - a
    return float((d + np.pi) % (np.pi * 2.0) - np.pi)


def eval_angle(a, gt_a, accept_inverted_angle):
    levels = np.array([0, 0, 0])
    level_thresholds = np.array([1., 5.0, 10.0])
    dist = np.abs(angle_diff(a, gt_a)) / np.pi * 180.
    if accept_inverted_angle:
        dist = np.minimum(dist, np.abs(angle_diff(a + np.pi, gt_a)) / np.pi * 180.)
    for idx, thresh in enumerate(level_thresholds):
        if dist < thresh:
            levels[idx] = 1
    return dist, levels


def eval_transform(t, gt_t, a, gt_a, accept_inverted_angle):
    _, levels_translation = eval_translation(t, gt_t)
    _, levels_angle = eval_angle(a, gt_a, accept_inverted_angle=accept_inverted_angle)
    return np.minimum(levels_translation, levels_angle)

In [99]:
def get_at_dist_measures(eval_measures, dist):
    return Namespace(
        corr_levels=eval_measures[dist]['corr_levels'].tolist(),
        corr_levels_translation=eval_measures[dist]['corr_levels_translation'].tolist(),
        mean_dist_translation=eval_measures[dist]['mean_dist_translation'],
        mean_sq_dist_translation=eval_measures[dist]['mean_sq_dist_translation'],
        corr_levels_angles=eval_measures[dist]['corr_levels_angles'].tolist(),
        mean_dist_angle=eval_measures[dist]['mean_dist_angle'],
        mean_sq_dist_angle=eval_measures[dist]['mean_sq_dist_angle'],
        num=eval_measures[dist]['num'],
    )

In [100]:
new_all_pred_translations = translate_transform_to_new_center_of_rotation(all_pred_translations, all_pred_angles, all_pred_centers, all_gt_pc1centers)
np.set_printoptions(precision=3, suppress=True)

# 0.INIT EMPTY CONTAINERS FOR THE MEASURES
# Diferents tracks
tracks = defaultdict(dict)

# Base dict for the measures 
empty_dict = {'corr_levels_translation': np.array([0, 0, 0], dtype=float), 'corr_levels_angles': np.array([0, 0, 0], dtype=float), 'corr_levels': np.array([0, 0, 0], dtype=float), 'mean_dist_translation': 0.0, 'mean_sq_dist_translation': 0.0, 'mean_dist_angle': 0.0, 'mean_sq_dist_angle': 0.0, 'num': 0}

# Big dict fot the measures
eval_measures = {
    'all': copy.deepcopy(empty_dict),
    '5m': copy.deepcopy(empty_dict),
    '10m': copy.deepcopy(empty_dict),
    '15m': copy.deepcopy(empty_dict),
    '20m': copy.deepcopy(empty_dict),
    'val': {
        'all': copy.deepcopy(empty_dict),
        '5m': copy.deepcopy(empty_dict),
        '10m': copy.deepcopy(empty_dict),
        '15m': copy.deepcopy(empty_dict),
        '20m': copy.deepcopy(empty_dict),
    },
    'test': {
        'all': copy.deepcopy(empty_dict),
        '5m': copy.deepcopy(empty_dict),
        '10m': copy.deepcopy(empty_dict),
        '15m': copy.deepcopy(empty_dict),
        '20m': copy.deepcopy(empty_dict),
    },
}

# Check 
per_transform_info = []

# 1.COMPUTE EVALUATIONS FOR EACH OBSERVATION
for idx, val_idx, translation, gt_translation, pred_angle, gt_angle, gt_pc1center in zip([x for x in range(len(val_idxs))], val_idxs, new_all_pred_translations, all_gt_translations, all_pred_angles, all_gt_angles, all_gt_pc1centers):
    
    # Load JSON (what for?)
    meta = json.load(open(f'{base_gt_dir}/meta/{str(val_idx).zfill(8)}.json', 'r'))

    # Depending on the track is test or it isn't
    is_test = 'trackids' in meta and meta['trackids'][0] in [2, 6, 7, 8, 10]
        
    # Translation evaluation
    dist_transl, levels_transl = eval_translation(translation, gt_translation)
    # Angle evaluation
    dist_angle, levels_angle = eval_angle(pred_angle, gt_angle, accept_inverted_angle=accept_inverted_angle)
    # What does this do?
    levels = eval_transform(translation, gt_translation, pred_angle, gt_angle, accept_inverted_angle=accept_inverted_angle)
    
    # What are sets?
    for _set in ['both', 'val', 'test']:
        if dist_transl > 10000:
            continue
        node = eval_measures
        if _set in ['val', 'test']:
            node = eval_measures[_set]
            if (_set == 'test') != is_test:
                continue
        for key in ['all', '5m', '10m', '15m', '20m']:
            centroid_distance = np.linalg.norm(gt_pc1center)
            if key == '5m' and centroid_distance > 5.:
                continue
            if key == '10m' and centroid_distance > 10.:
                continue
            if key == '15m' and centroid_distance > 15.:
                continue
            if key == '20m' and centroid_distance > 20.:
                continue
            node[key]['num'] += 1
            node[key]['corr_levels_translation'] += levels_transl
            node[key]['mean_dist_translation'] += dist_transl
            node[key]['mean_sq_dist_translation'] += dist_transl * dist_transl

            node[key]['corr_levels_angles'] += levels_angle
            node[key]['mean_dist_angle'] += dist_angle
            node[key]['mean_sq_dist_angle'] += dist_angle * dist_angle

            node[key]['corr_levels'] += levels

    if detailed_eval:
        per_transform_info.append([levels, dist_transl, dist_angle])


for _set in ['both', 'val', 'test']:
    node = eval_measures
    if _set in ['val', 'test']:
        node = eval_measures[_set]
    for key in ['all', '5m', '10m', '15m', '20m']:
        num_predictions = float(node[key]['num'])
        if node[key]['num'] == 0:
            num_predictions = 1e-20  # make numbers really large, indicates eval is not valid
        node[key]['corr_levels_translation'] /= num_predictions
        node[key]['mean_dist_translation'] /= num_predictions
        node[key]['mean_sq_dist_translation'] = np.sqrt(node[key]['mean_sq_dist_translation'] / num_predictions)
        node[key]['corr_levels_angles'] /= num_predictions
        node[key]['mean_dist_angle'] /= num_predictions
        node[key]['mean_sq_dist_angle'] = np.sqrt(node[key]['mean_sq_dist_angle'] / num_predictions)
        node[key]['corr_levels'] /= num_predictions

reg_eval_measures = np.array([0, 0], dtype=float)
for idx, file_idx in enumerate(val_idxs):
    meta = json.load(open(f'{base_gt_dir}/meta/{str(file_idx).zfill(8)}.json', 'r'))
    if 'seq' in meta:
        seq = meta['seq']
        trackid = meta['trackids'][0]
        frame1, frame2 = meta['frames']
        intermediate_trackid = seq * 10000000 + trackid * 10000
        pred_translation = all_pred_translations[idx]
        time_passed = 0.1
        tracks[intermediate_trackid][frame2] = (pred_translation, time_passed)

#if len(tracks) > 0:
#    velocities = process_velocities(tracks, eval_dir, avg_window)
#    velocities
#  print(velocities)

eval_dict = Namespace(
    corr_levels=eval_measures['all']['corr_levels'].tolist(),
    corr_levels_translation=eval_measures['all']['corr_levels_translation'].tolist(),
    mean_dist_translation=eval_measures['all']['mean_dist_translation'],
    mean_sq_dist_translation=eval_measures['all']['mean_sq_dist_translation'],
    corr_levels_angles=eval_measures['all']['corr_levels_angles'].tolist(),
    mean_dist_angle=eval_measures['all']['mean_dist_angle'],
    mean_sq_dist_angle=eval_measures['all']['mean_sq_dist_angle'],
    num=eval_measures['all']['num'],
    eval_5m=get_at_dist_measures(eval_measures, '5m'),
    eval_10m=get_at_dist_measures(eval_measures, '10m'),
    eval_15m=get_at_dist_measures(eval_measures, '15m'),
    eval_20m=get_at_dist_measures(eval_measures, '20m'),
    val=Namespace(
        corr_levels=eval_measures['val']['all']['corr_levels'].tolist(),
        corr_levels_translation=eval_measures['val']['all']['corr_levels_translation'].tolist(),
        mean_dist_translation=eval_measures['val']['all']['mean_dist_translation'],
        mean_sq_dist_translation=eval_measures['val']['all']['mean_sq_dist_translation'],
        corr_levels_angles=eval_measures['val']['all']['corr_levels_angles'].tolist(),
        mean_dist_angle=eval_measures['val']['all']['mean_dist_angle'],
        mean_sq_dist_angle=eval_measures['val']['all']['mean_sq_dist_angle'],
        num=eval_measures['val']['all']['num'],
        eval_5m=get_at_dist_measures(eval_measures['val'], '5m'),
        eval_10m=get_at_dist_measures(eval_measures['val'], '10m'),
        eval_15m=get_at_dist_measures(eval_measures['val'], '15m'),
        eval_20m=get_at_dist_measures(eval_measures['val'], '20m'),
    ),
    test=Namespace(
        corr_levels=eval_measures['test']['all']['corr_levels'].tolist(),
        corr_levels_translation=eval_measures['test']['all']['corr_levels_translation'].tolist(),
        mean_dist_translation=eval_measures['test']['all']['mean_dist_translation'],
        mean_sq_dist_translation=eval_measures['test']['all']['mean_sq_dist_translation'],
        corr_levels_angles=eval_measures['test']['all']['corr_levels_angles'].tolist(),
        mean_dist_angle=eval_measures['test']['all']['mean_dist_angle'],
        mean_sq_dist_angle=eval_measures['test']['all']['mean_sq_dist_angle'],
        num=eval_measures['test']['all']['num'],
        eval_5m=get_at_dist_measures(eval_measures['test'], '5m'),
        eval_10m=get_at_dist_measures(eval_measures['test'], '10m'),
        eval_15m=get_at_dist_measures(eval_measures['test'], '15m'),
        eval_20m=get_at_dist_measures(eval_measures['test'], '20m'),
    ),
    reg_eval=Namespace(fitness=reg_eval_measures[0], inlier_rmse=reg_eval_measures[1]),
    #  num=len(val_idxs),
    mean_time=mean_time)

os.makedirs(eval_dir, exist_ok=True)
filename = f'{eval_dir}/eval{"_180" if accept_inverted_angle else ""}.json'
if os.path.isfile(filename):
    datestr_now = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    copyfile(filename, f'{filename[:-5]}_{datestr_now}.json')
    if mean_time == 0:
        prev_eval_dict = json.load(open(filename, 'r'))
        if 'mean_time' in prev_eval_dict:
            eval_dict.__dict__['mean_time'] = prev_eval_dict['mean_time']

    with open(filename, 'w') as fhandle:
        json.dump(ns_to_dict(eval_dict), fhandle)



In [105]:
eval_dict.

Namespace(corr_levels=[0.06606566200215286, 0.5060548977395048, 0.7459634015069968], corr_levels_angles=[0.5670075349838536, 0.6922766415500539, 0.8203713670613563], corr_levels_translation=[0.08032831001076426, 0.58745963401507, 0.7672228202368138], eval_10m=Namespace(corr_levels=[0.13953488372093023, 0.8057455540355677, 0.957592339261286], corr_levels_angles=[0.7701778385772914, 0.8714090287277702, 0.9671682626538988], corr_levels_translation=[0.16552667578659372, 0.8864569083447332, 0.9671682626538988], mean_dist_angle=6.833718817428427, mean_dist_translation=0.06107330318220551, mean_sq_dist_angle=31.083310430242665, mean_sq_dist_translation=0.10073639638011792, num=731), eval_15m=Namespace(corr_levels=[0.13473520249221183, 0.8029595015576324, 0.9493769470404985], corr_levels_angles=[0.7538940809968847, 0.8753894080996885, 0.9641744548286605], corr_levels_translation=[0.16433021806853582, 0.8800623052959502, 0.955607476635514], mean_dist_angle=7.45575269529974, mean_dist_translatio